In [6]:
import os, pprint
import chainlit as cl
from langchain import PromptTemplate
from langchain.llms import CTransformers
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

In [2]:
DATA_PATH = 'knowledge/docs/'
DB_FAISS_PATH = 'knowledge/vectorstore/db_faiss'

In [3]:
def create_vector_db():
    if os.path.exists(DB_FAISS_PATH):
        print('FAISS DB already exists')

    else:
        loader = DirectoryLoader(
                                DATA_PATH,
                                glob='*.pdf',
                                loader_cls=PyPDFLoader
                                )

        documents = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                                    chunk_overlap=50)
        texts = text_splitter.split_documents(documents)

        embeddings = HuggingFaceEmbeddings(
                                            model_name='sentence-transformers/all-MiniLM-L6-v2',
                                            model_kwargs={'device': 'mps'}
                                            )

        db = FAISS.from_documents(texts, embeddings)
        db.save_local(DB_FAISS_PATH)

        print('FAISS DB created')

create_vector_db()

2023-09-28 20:19:59 - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/1zuu/anaconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


Batches:   0%|          | 0/67 [00:00<?, ?it/s]

2023-09-28 20:20:37 - Loading faiss.
2023-09-28 20:20:37 - Successfully loaded faiss.
FAISS DB created


In [4]:
custom_prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

def set_custom_prompt():
    """
    Prompt template for QA retrieval for each vectorstore
    """
    prompt = PromptTemplate(
                            template=custom_prompt_template,
                            input_variables=['context', 'question']
                            )
    return prompt

def retrieval_qa_chain(llm, prompt, db):
    qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type='stuff',
                                       retriever=db.as_retriever(search_kwargs={'k': 2}),
                                       return_source_documents=True,
                                       chain_type_kwargs={'prompt': prompt}
                                       )
    return qa_chain

def load_llm():
    llm = CTransformers(
                        model = "TheBloke/Llama-2-7B-Chat-GGML",
                        model_type="llama",
                        max_new_tokens = 512,
                        temperature = 0.5
                        )
    return llm

def qa_bot():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                       model_kwargs={'device': 'mps'})
    db = FAISS.load_local(DB_FAISS_PATH, embeddings)
    llm = load_llm()
    qa_prompt = set_custom_prompt()
    qa = retrieval_qa_chain(llm, qa_prompt, db)

    return qa

def final_result(query):
    qa_result = qa_bot()
    response = qa_result({'query': query})
    return response

In [12]:
response = final_result('What is Buddha-Dhamma?')
pprint.pprint(response)

2023-09-28 20:25:50 - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'query': 'What is Buddha-Dhamma?',
 'result': 'Buddha-Dhamma refers to the moral and philosophical system taught '
           'by the Buddha, which includes his teachings on ethics, morality, '
           'and spiritual enlightenment.',
 'source_documents': [Document(page_content='to the learned writers on their laudable efforts to enlighten the \nreaders on the Buddha-Dhamma.\nThis new treatise is another humble attempt made by a \nmember of the Order of the Sangha, based on the Pāli Texts, \ncommentaries, and traditions prevailing in Buddhist countries, \nespecially in Ceylon.\nThe first part of the book deals with the Life of the Buddha, \nthc second with the Dhamma, the Pāli term for His Doctrine.\n*\nThe Buddha-Dhamma is a moral and philosophical system', metadata={'source': 'knowledge/docs/buddha-teachingsurw6.pdf', 'page': 6}),
                      Document(page_content='Truth. He is also called Dhammassāmi as He is the Lord of the \nDhamma (Doctrine).\nAs the Buddha Himself s